In [1]:
import pandas as pd
import glob
import os
from geopy.geocoders import Nominatim
from geopy.distance import great_circle
import folium

In [2]:
# 데이터 전처리

In [5]:
path = 'DataSets/'
allFiles = glob.glob(os.path.join(path+"*.csv"))

df = pd.DataFrame()
for file in allFiles:
    
    temp = pd.read_csv(file, engine='python', encoding='utf-8')
    df = pd.concat([df, temp])

In [6]:
df.columns

Index(['상가업소번호', '상호명', '지점명', '상권업종대분류코드', '상권업종대분류명', '상권업종중분류코드',
       '상권업종중분류명', '상권업종소분류코드', '상권업종소분류명', '표준산업분류코드', '표준산업분류명', '시도코드',
       '시도명', '시군구코드', '시군구명', '행정동코드', '행정동명', '법정동코드', '법정동명', '지번코드',
       '대지구분코드', '대지구분명', '지번본번지', '지번부번지', '지번주소', '도로명코드', '도로명', '건물본번지',
       '건물부번지', '건물관리번호', '건물명', '도로명주소', '구우편번호', '신우편번호', '동정보', '층정보',
       '호정보', '경도', '위도', 'Unnamed: 0'],
      dtype='object')

In [8]:
df.to_csv('DataSets/DataFrame.csv')

In [4]:
dataset = df[['상가업소번호','상호명','지점명',
              '상권업종대분류명', '상권업종중분류명','상권업종소분류명'
              '시도명', '시군구명', '행정동명',
              '위도', '경도']]

In [5]:
df_coffee = dataset[(dataset['상권업종중분류명']=='커피점/카페')]
df_seoul_starbucks = df_coffee[df_coffee['상호명'].str.contains('스타벅스')]

In [24]:
class CountByWGS84:

    def __init__(self, df, lat, lon, dist=1):

        self.df = df
        self.lat = lat
        self.lon = lon
        self.dist = dist
    
    
    def filter_by_radius(self):
        lat_min = self.lat - 0.01 * self.dist
        lat_max = self.lat + 0.01 * self.dist

        lon_min = self.lon - 0.015 * self.dist
        lon_max = self.lon + 0.015 * self.dist

        self.points = [[lat_min, lon_min], [lat_max, lon_max]]

        filterResult = self.df.loc[
            (self.df['위도'] > lat_min) &
            (self.df['위도'] < lat_max) &
            (self.df['경도'] > lon_min) &
            (self.df['경도'] < lon_max)
        ]
        filterResult.index = range(len(filterResult))
        print(filterResult)


        # 기준 좌표 포인트
        center = (self.lat, self.lon)

        result = pd.DataFrame()
        for index, row in filterResult.iterrows():
            # 개별 좌표 포인트
            point = (row['위도'], row['경도'])
            d = great_circle(center, point).kilometers
            if d <= self.dist:
                result = pd.concat([result, filterResult.iloc[index, :].to_frame().T])
        result.index = range(len(result))
        return result

    def plot_by_radius(self, df):


        m = folium.Map(location=[self.lat, self.lon], zoom_start=14)

        for idx, row in df.iterrows():

            lat_ = row['위도']
            lon_ = row['경도']

            folium.Marker(location=[lat_, lon_],
                          radius=15,
                          tooltip=row['지점명']).add_to(m)

        folium.Circle(radius=dist * 1000,
                      location=[lat, lon],
                      color="#ff7800",
                      fill_color='#ffff00',
                      fill_opacity=0.2
                      ).add_to(m)

        return m._repr_html_()

In [25]:


# 파라미터 설정
# 35.1613737 129.1479813
# n = input("현재 위치를 적어주세요:")
# app = Nominatim(user_agent='tutorial')
# location = app.geocode('{n}'.format(n=n))
# print(location.latitude, location.longitude)

lat = 35.1613737
lon = 129.1479813
dist = 1
# 반경 집계 인스턴스 생성
cbw = CountByWGS84(df_seoul_starbucks, lat, lon, dist)

In [26]:
# 반경 범위 내 데이터 필터링
result_radius = cbw.filter_by_radius()


plot_2 = cbw.plot_by_radius(result_radius)
plot_2

        상호명       지점명 상권업종대분류명 상권업종중분류명    시도명  시군구명 행정동명         위도  \
0      스타벅스     하버타운점       음식   커피점/카페  부산광역시  해운대구  우1동  35.159707   
1      스타벅스      벡스코점       음식   커피점/카페  부산광역시  해운대구  우2동  35.169079   
2      스타벅스      해운대점       음식   커피점/카페  부산광역시  해운대구  우1동  35.163330   
3  스타벅스해운대점      해운대점       음식   커피점/카페  부산광역시  해운대구  중1동  35.160177   
4      스타벅스      해운대점       음식   커피점/카페  부산광역시  해운대구  중1동  35.160465   
5      스타벅스       NaN       음식   커피점/카페  부산광역시  해운대구  우3동  35.155055   
6      스타벅스    더해운대R점       음식   커피점/카페  부산광역시  해운대구  우1동  35.158824   
7      스타벅스  신세계마린시티점       음식   커피점/카페  부산광역시  해운대구  우3동  35.157057   
8      스타벅스   제니스스퀘어점       음식   커피점/카페  부산광역시  해운대구  우3동  35.157224   
9      스타벅스     베네시티점       음식   커피점/카페  부산광역시  해운대구  우3동  35.158186   

           경도  
0  129.159540  
1  129.136039  
2  129.159613  
3  129.162203  
4  129.162273  
5  129.145426  
6  129.152865  
7  129.143357  
8  129.144841  
9  129.150749  


'<div style="width:100%;"><div style="position:relative;width:100%;height:0;padding-bottom:60%;"><span style="color:#565656">Make this Notebook Trusted to load map: File -> Trust Notebook</span><iframe srcdoc="&lt;!DOCTYPE html&gt;\n&lt;head&gt;    \n    &lt;meta http-equiv=&quot;content-type&quot; content=&quot;text/html; charset=UTF-8&quot; /&gt;\n    \n        &lt;script&gt;\n            L_NO_TOUCH = false;\n            L_DISABLE_3D = false;\n        &lt;/script&gt;\n    \n    &lt;style&gt;html, body {width: 100%;height: 100%;margin: 0;padding: 0;}&lt;/style&gt;\n    &lt;style&gt;#map {position:absolute;top:0;bottom:0;right:0;left:0;}&lt;/style&gt;\n    &lt;script src=&quot;https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.js&quot;&gt;&lt;/script&gt;\n    &lt;script src=&quot;https://code.jquery.com/jquery-1.12.4.min.js&quot;&gt;&lt;/script&gt;\n    &lt;script src=&quot;https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js&quot;&gt;&lt;/script&gt;\n    &lt;script